# EventVestor: Credit Facility

In this notebook, we'll take a look at EventVestor's *Credit Facility* dataset, available on the [Quantopian Store](https://www.quantopian.com/store). This dataset spans January 01, 2007 through the current day, and documents financial events covering new or extended credit facilities.

### Blaze
Before we dig into the data, we want to tell you about how  you generally access Quantopian Store data sets. These datasets are available through an API service known as [Blaze](http://blaze.pydata.org). Blaze provides the Quantopian user with a convenient interface to access very large datasets.

Blaze provides an important function for accessing these datasets. Some of these sets are many millions of records. Bringing that data directly into Quantopian Research directly just is not viable. So Blaze allows us to provide a simple querying interface and shift the burden over to the server side.

It is common to use Blaze to reduce your dataset in size, convert it over to Pandas and then to use Pandas for further computation, manipulation and visualization.

Helpful links:
* [Query building for Blaze](http://blaze.pydata.org/en/latest/queries.html)
* [Pandas-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-pandas.html)
* [SQL-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-sql.html).

Once you've limited the size of your Blaze object, you can convert it to a Pandas DataFrames using:
> `from odo import odo`  
> `odo(expr, pandas.DataFrame)`

### Free samples and limits
One other key caveat: we limit the number of results returned from any given expression to 10,000 to protect against runaway memory usage. To be clear, you have access to all the data server side. We are limiting the size of the responses back from Blaze.

There is a *free* version of this dataset as well as a paid one. The free one includes about three years of historical data, though not up to the current day.

With preamble in place, let's get started:

In [3]:
# import the dataset
from quantopian.interactive.data.eventvestor import credit_facility
# or if you want to import the free dataset, use:
# from quantopian.data.eventvestor import credit_facility_free

# import data operations
from odo import odo
# import other libraries we will use
import pandas as pd

In [4]:
# Let's use blaze to understand the data a bit using Blaze dshape()
credit_facility.dshape

dshape("""var * {
  event_id: ?float64,
  asof_date: datetime,
  trade_date: ?datetime,
  symbol: ?string,
  event_type: ?string,
  event_headline: ?string,
  credit_amount: ?float64,
  credit_units: ?string,
  event_rating: ?float64,
  timestamp: datetime,
  sid: ?int64
  }""")

In [5]:
# And how many rows are there?
# N.B. we're using a Blaze function to do this, not len()
credit_facility.count()

8000

In [6]:
# Let's see what the data looks like. We'll grab the first three rows.
credit_facility[:3]

,event_id,asof_date,trade_date,symbol,event_type,event_headline,credit_amount,credit_units,event_rating,timestamp,sid
0,78219,2007-01-03,2007-01-03,NVLS,Credit Facility,Novellus Signs $150M Credit Facility,150,$M,1,2007-01-04,5509
1,961784,2007-01-04,2007-01-04,NAV,Credit Facility,Navistar International Gets $1.3B Credit Facil...,1300,$M,1,2007-01-05,5199
2,145867,2007-01-10,2007-01-10,CCI,Credit Facility,Crown Castle Signs $250M Revolving Credit Faci...,250,$M,1,2007-01-11,19258


Let's go over the columns:
- **event_id**: the unique identifier for this event.
- **asof_date**: EventVestor's timestamp of event capture.
- **trade_date**: for event announcements made before trading ends, trade_date is the same as event_date. For announcements issued after market close, trade_date is next market open day.
- **symbol**: stock ticker symbol of the affected company.
- **event_type**: this should always be *Credit Facility/Credit facility*.
- **event_headline**: a brief description of the event
- **credit_amount**: the amount of credit_units being availed
- **credit_units**: the units for credit_amount: currency or other value. Most commonly in millions of USD.
- **event_rating**: this is always 1. The meaning of this is uncertain.
- **timestamp**: this is our timestamp on when we registered the data.
- **sid**: the equity's unique identifier. Use this instead of the symbol.

We've done much of the data processing for you. Fields like `timestamp` and `sid` are standardized across all our Store Datasets, so the datasets are easy to combine. We have standardized the `sid` across all our equity databases.

We can select columns and rows with ease. Below, we'll fetch all events in which \$ 200M of credit was availed.

In [7]:
twohundreds = credit_facility[(credit_facility.credit_amount==200) & (credit_facility.credit_units=="$M")]
# When displaying a Blaze Data Object, the printout is automatically truncated to ten rows.
twohundreds.sort('timestamp')

,event_id,asof_date,trade_date,symbol,event_type,event_headline,credit_amount,credit_units,event_rating,timestamp,sid
0,910312,2007-01-16,2007-01-16,CPNO,Credit Facility,Copano Energy Completes Amendment of $200M Cre...,200,$M,1,2007-01-17,26783
1,965519,2007-01-22,2007-01-22,NAV,Credit Facility,Navistar International Raises Credit Facility ...,200,$M,1,2007-01-23,5199
2,133561,2007-03-13,2007-03-13,SIAL,Credit Facility,Sigma-Aldrich Announces New European Credit Fa...,200,$M,1,2007-03-14,6872
3,131670,2007-05-04,2007-05-04,LEG,Credit Facility,Leggett & Platt Increases Multi-Currency Credi...,200,$M,1,2007-05-05,4415
4,125820,2007-05-31,2007-05-31,ABI,Credit Facility,Applera Signs $200M Credit Agreement,200,$M,1,2007-06-01,25270
5,961810,2007-06-15,2007-06-15,NAV,Credit Facility,Navistar International Unit Gets $200M Credit ...,200,$M,1,2007-06-16,5199
6,78520,2007-07-25,2007-07-25,JBL,Credit Facility,Jabil Increases Credit Facility to $1B,200,$M,1,2007-07-26,8831
7,91869,2007-09-17,2007-09-17,AYE,Credit Facility,Allegheny Increases Credit Facility to $400M,200,$M,1,2007-09-18,17618
8,93042,2007-09-18,2007-09-18,AIV,Credit Facility,AIMCO Increases Credit Facility by $200M,200,$M,1,2007-09-19,11598
9,140305,2007-10-04,2007-10-04,NTRI,Credit Facility,Nutri/System Establishes $200M Credit Facility,200,$M,1,2007-10-05,21697


Finally, suppose we want a DataFrame of that data, but we only want the symbol, timestamp, and event headline:

In [8]:
twohundred_df = odo(twohundreds, pd.DataFrame)
reduced = twohundred_df[['symbol','event_headline','timestamp']]
# When printed: pandas DataFrames display the head(30) and tail(30) rows, and truncate the middle.
reduced

,symbol,event_headline,timestamp
0,CPNO,Copano Energy Completes Amendment of $200M Cre...,2007-01-17 00:00:00
1,NAV,Navistar International Raises Credit Facility ...,2007-01-23 00:00:00
2,SIAL,Sigma-Aldrich Announces New European Credit Fa...,2007-03-14 00:00:00
3,LEG,Leggett & Platt Increases Multi-Currency Credi...,2007-05-05 00:00:00
4,ABI,Applera Signs $200M Credit Agreement,2007-06-01 00:00:00
5,NAV,Navistar International Unit Gets $200M Credit ...,2007-06-16 00:00:00
6,JBL,Jabil Increases Credit Facility to $1B,2007-07-26 00:00:00
7,AYE,Allegheny Increases Credit Facility to $400M,2007-09-18 00:00:00
8,AIV,AIMCO Increases Credit Facility by $200M,2007-09-19 00:00:00
9,NTRI,Nutri/System Establishes $200M Credit Facility,2007-10-05 00:00:00
